In [ ]:
import cv2
from skimage import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import os
import skimage.exposure
from skimage import transform
from skimage.measure import shannon_entropy
import random
import re

from copy import deepcopy

In [ ]:
# DATA_FOLDER='/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/raw/Opera18Days_sorted/batch1/WT/panelE/Untreated/rep1/DCP1A/'#'/Users/nancy/PycharmProjects/MOmaps/example_raw_images'


BASE_DIR = os.path.join('/home','labs','hornsteinlab','Collaboration','MOmaps')
DATA_FOLDER = os.path.join(BASE_DIR,'input','images','raw','SpinningDisk', 'NOVA_d18_neurons_sorted')

In [ ]:
# break to 64 tiles of 128x128, resize to 100x100
def break_to_tiles(image, tile_size=128):

    # Get the dimensions of the image
    height, width = image.shape

    # Calculate the number of rows and columns for the tiles
    num_rows = height // tile_size
    num_cols = width // tile_size
    print(f"shannon_entropy all {shannon_entropy(image)}")
    # Iterate through the rows and columns to display each tile
    for row in range(num_rows):
        for col in range(num_cols):
            
            fig, axs = plt.subplots(figsize=(2,2))
            
            # Define the coordinates for the current tile
            x1 = col * tile_size
            x2 = (col + 1) * tile_size
            y1 = row * tile_size
            y2 = (row + 1) * tile_size
            # Extract the current tile
            tile = image[y1:y2, x1:x2]
            
            # Resize the 128x128 tile to 100x100
            #tile_resized = transform.resize(tile, (100, 100), anti_aliasing=True, preserve_range=True)
            #tile_resized = cv2.resize(tile, (100, 100), interpolation=cv2.INTER_LINEAR)
            #tile_resized = tile[:100, :100]

            
            # Display the tile using Matplotlib
            plt.imshow(tile, cmap='gray', vmin=0, vmax=1)
            plt.axis('off')
            #plt.title(f'Tile ({row}, {col}) shannon_entropy tile {shannon_entropy(tile)} {shannon_entropy(tile_resized)} {tile.std()} {tile_resized.std()}')
            plt.title(f'Tile ({row}, {col}) shannon_entropy tile {shannon_entropy(tile)} {tile.std()}')
            plt.show()

In [ ]:

def normalize_piecewise(img_current_channel, to_show):
    """Normalize image by the dynamic scale giving emphasie to the signal area """
    try: 
        Piecewise_norm = Piecewise_normalization()  #static class - so no need to create instance
        Piecewise_norm.create_peicewise_lut()
                    
        img_current_channel = Piecewise_norm.map(ipnut_image = img_current_channel, show = to_show)
    
        return img_current_channel
    except Exception as ex:
            raise f"Can't convert in normalize_piecewise" 


class Piecewise_normalization(object): #static class
    
    lut = None
    index = None
    
    def create_peicewise_lut(self):
        
        try:
        
            if not(self.lut is None):
                return #lut already exist
            
            #reaching here - this is 1st time i.e. sigelton
            
            max_uint16 = np.iinfo(np.uint16).max + 1
            max_uint8 = np.iinfo(np.uint8).max + 1
            
            input_regions_zones = [0,401,4001,8001, max_uint16]
            output_regions_zones = [0,220, max_uint8, max_uint8]
            contrasts = [1.00,1.00]  #[1,1]
            range = input_regions_zones[2] - input_regions_zones[1]
            range2 = output_regions_zones[1] - output_regions_zones[0]
            
            shrinkage_factor = np.zeros(2)
            shrinkage_factor[0] = range/range2
            
            range = input_regions_zones[3] - input_regions_zones[2]
            range2 = output_regions_zones[2] - output_regions_zones[1]
            
            shrinkage_factor[1] = range/range2
            
            
            region_start = np.zeros(input_regions_zones[1], dtype = np.uint32)
            region_end = np.ones(input_regions_zones[4]-input_regions_zones[3], dtype = np.uint32) * 255
            region1 = np.zeros(input_regions_zones[2]-input_regions_zones[1], dtype = np.uint32)
            region2 = np.zeros(input_regions_zones[3]-input_regions_zones[2], dtype = np.uint32)
            
            #print('region1')
            #print('i,input,output')
            for i in np.arange(input_regions_zones[1],input_regions_zones[2],1):
                x = i-input_regions_zones[1]
                region1[x] = np.round(contrasts[0] *(x/ shrinkage_factor[0]) + output_regions_zones[0])
                #print(x, i, region1[x])
            
            #print('region2')
            #print('i,input,output')
            for i in np.arange(input_regions_zones[2],input_regions_zones[3],1):
                x = i-input_regions_zones[2]
                region2[x] = np.round(contrasts[1] *(x/ shrinkage_factor[1]) + output_regions_zones[1])
                #print(x, i, region2[x])
        
            self.lut = np.concatenate((region_start, region1, region2, region_end), axis=None) 
            
            # Ensure table is 16-bit
            self.lut = self.lut.astype(np.uint16)
            self.index = np.arange(start = input_regions_zones[0], stop = input_regions_zones[-1])
        except Exception as ex:
            print('bad create_peicewise_lut')
    
    def map(self,ipnut_image, show):
        try:
            # Now just index into this with the intensities to get the output
            transformed_image =  self.lut[ipnut_image].astype('uint8')
            transformed_npy = transformed_image.astype('float')/(256.0)
            
            return transformed_npy
        
        except Exception as ex:
            print('bad LUT in map', ex)
            
    
    def show_mapping(self, path2save):
        
        plt.plot(self.lut[0:10000])
        plt.xlabel('16b')
        plt.ylabel('8b')
        plt.savefig(path2save+'LUT.png') 
        plt.close("all")
        
        #np.save(path2save+'LUT.csv', self.lut)
        np.savetxt(path2save+'LUT.csv', np.c_[self.index, self.lut] , delimiter=',')
    
def plots(ax, i, img_scaled, nucleus_scaled, title):

    print(f"\n{title} min: {img_scaled.min().round(2)}, max: {img_scaled.max().round(2)},\nmean: {img_scaled.mean().round(2)}, std: {img_scaled.std().round(2)}")

    # Plot scaled image
    ax[i,0].imshow(img_scaled, cmap='gray')
    ax[i,0].set_title(title, color='red')
    ax[i,1].imshow(nucleus_scaled, cmap='gray')
    ax[i,1].set_title('Nucleus', color='red')
    #fig.text(0.7, 0.3, 
    #             f"min: {img_scaled.min().round(2)}, max: {img_scaled.max().round(2)},\nmean: {img_scaled.mean().round(2)}, std: {img_scaled.std().round(2)}", 
    #             fontsize = 16)
    ax[i,0].set_xticks([])
    ax[i,0].set_yticks([])

    # Plot scaled pixel histogram
    ax[i,2].hist(img_scaled.ravel(), bins=10, color='black', fc='k', ec='k')
    ax[i,2].set_title(title + ' pixel histogram', color='red')
    return ax
    
    
def collect_files_from_subfolders(root_path, folder_name, cell_lines=None, conditions=None):
    collected_files = []

    def contains_any_substring(main_string, substrings):
        return any(substring in main_string for substring in substrings)
    
    for dirpath, dirnames, filenames in os.walk(root_path):
        valid = True
        valid &= True if (cell_lines is None or len(cell_lines) == 0) else contains_any_substring(dirpath, cell_lines)
        valid &= True if conditions is None or len(conditions) == 0 else contains_any_substring(dirpath, conditions)
        
        if valid and os.path.basename(dirpath) == folder_name:
            for filename in filenames:
                collected_files.append(os.path.join(dirpath, filename))
    
    return collected_files

def get_markers(root_path):
    markers = []
    for dirpath, dirnames, filenames in os.walk(root_path):
        if len(filenames) > 0 and os.path.splitext(filenames[0])[1] == '.tiff':
            markers.append(os.path.basename(dirpath))
    return np.unique(markers)
            

def sample_files(root_path, folder_name, cell_lines=None, conditions=None, n=10):
    all_files = collect_files_from_subfolders(root_path, folder_name, cell_lines=cell_lines, conditions=conditions)
    
    if len(all_files) < n:
        raise ValueError(f"Not enough files to sample from. Found {len(all_files)} files.")
    
    sampled_files = random.sample(all_files, n)
    return sampled_files

def plot_target_and_dapi_rescaled(indx, img, dapi, mn, mx):
    vmin, vmax = np.percentile(img, q=(mn, mx))

    img_scaled = skimage.exposure.rescale_intensity(
        img,
        in_range=(vmin, vmax),
        out_range=np.float32
    )

    vmin, vmax = np.percentile(dapi, q=(mn, mx))

    img_scaled_dapi = skimage.exposure.rescale_intensity(
        dapi,
        in_range=(vmin, vmax),
        out_range=np.float32
    )

    plots(ax, indx, img_scaled,img_scaled_dapi, title=f'{mn},{mx}')

In [ ]:

######## SAFE ZONE TO CHANGE ############

number_of_images_per_marker = 10
ranges = zip([0.5,  0.5,  0.5,  0.5,  0.1,   0.1,    0.1,    1,    2],
                         [99.9, 99.0, 98.5, 98.0, 99.9, 99.0,    98.0,  99.9, 99.9])
markers = ['G3BP1']
# markers = ['AGO2', 'ANXA11', 'CD41', 'CLTC', 'Calreticulin', 'DAPI', 'DCP1A', 'FMRP', 'FUS',
#            'G3BP1', 'GM130', 'HNRNPA1', 'KIF5A', 'LAMP1', 'NCL', 'NEMO', 'NONO', 'PEX14',
#            'PML', 'PSD95', 'PSPC1', 'PURA', 'Phalloidin', 'SNCA', 'SQSTM1', 'TDP43',
#            'TOMM20', 'Tubulin', 'VDAC1', 'mitotracker']

cell_lines = ['WT']
conditions = ['Untreated']

############# END OF SAFE ZONE ##########


######################################################

markers = np.asarray(markers)

ranages_copy = deepcopy(ranges)

n_ranges = len(list(ranages_copy))
               
# markers = get_markers(DATA_FOLDER)
print(markers)
print(f"Data folder: {DATA_FOLDER}")

for marker in markers:
    print("----------------------------------------------")
    print(f"------------------ MARKER = {marker} ------------------")
    
    image_paths = sample_files(DATA_FOLDER, marker, cell_lines=cell_lines, conditions=conditions, n=number_of_images_per_marker)

    for image_path in image_paths:
        if image_path[-4:]=='tiff' or image_path[-3:]=='tif':
            print(image_path)
            dapi_path = image_path.replace(marker, 'DAPI')
            # dapi_path = re.sub('ch\d\d', 'ch01', dapi_path)
            dapi_path = re.sub('w[2-4]conf.*_s', 'w1confDAPI_s', dapi_path)
            
            
            # Load image
            #raw_img = io.imread(os.path.join(DATA_FOLDER, image_path))
            #raw_img = cv2.imread(os.path.join(DATA_FOLDER, image_path))
            raw_img = cv2.imread(image_path, cv2.IMREAD_ANYDEPTH) #used to be IMREAD_GRAYSCALE
            print(raw_img.shape, raw_img.dtype)
            
            raw_img_dapi = cv2.imread(dapi_path, cv2.IMREAD_ANYDEPTH) #used to be IMREAD_GRAYSCALE
            

            # Take one of the chanells (all the same, confirmed with Lena)
            #img = np.array(raw_img[:,:,0], dtype=np.float64)
            img = raw_img
            dapi = raw_img_dapi
            
            rows, cols = n_ranges+1, 3
            fig, ax = plt.subplots(rows,cols, figsize=(20, 20))
            # Plot raw image
            plots(ax, 0, img, dapi, title='Raw image ')

            # Scaling and transform #############################################

            
            ranages_copy = deepcopy(ranges)
            for indx, (mn, mx) in enumerate(ranages_copy):
                plot_target_and_dapi_rescaled(indx+1, img, dapi, mn, mx)
            

            # Cosmetics - add frame
            fig.suptitle(f"{os.path.basename(os.path.dirname(image_path))}\n{image_path.replace(DATA_FOLDER, '')}\n\n")
            fig.subplots_adjust(top=0.9, bottom=0.15, left=0.2, hspace=0.1)

            fig.patch.set_linewidth(10)
            fig.patch.set_edgecolor('cornflowerblue')

            fig.tight_layout()
            plt.show()
            
            # DAPI


In [ ]:
print("Done!!")